In [1]:
import pandas as pd

In [2]:
import pyarrow.parquet as pq
from sqlalchemy import Date as sadt
from io import StringIO
from glob import glob

In [3]:
data_loc = "D:\\DATA_DUMP\\DATA_QUALITY\\"
dq_main = glob(data_loc+"\\*od_dq_nhm*")

In [4]:
df = pd.read_parquet(dq_main)

In [5]:
# Load the data into Postgresql 
# ===========================================

df["curr_date"] = [x.date() for x in pd.to_datetime(df["curr_date"], format="%Y-%m-%d")]
df["ord_date"] = [x.date() for x in pd.to_datetime(df["ord_date"], format="%Y-%m-%d")]

In [6]:
for col in df.columns:
  if col not in ["curr_date", "ord_date", "seller_np"]:
    df[col] = df[col].astype(int)

In [ ]:
df.to_sql(
	name="od_dq_nhm",
	con= conn_str,
	index=False,
	if_exists='append',
	schema="data_sanity"
)

In [ ]:
# # Connection information
# conn_info = {
#     'dbname': 'db_stage',
#     'user': 'postgres',
#     'password': 'password',
#     'host': '172.31.41.128',
#     'port': '5432'
# }

In [ ]:
# import pyarrow.parquet as pq
# import psycopg
# from io import StringIO

# def dump_parquet_to_postgresql(parquet_file_path, table_name, conn_info):
#     table = pq.read_table(parquet_file_path)
#     df = table.to_pandas()
#     try:
#         with psycopg.connect(**conn_info) as conn:
#             with conn.cursor() as cur:
#                 # Use COPY to dump data into PostgreSQL
#                 buffer = StringIO()
#                 df.to_csv(buffer, index=False, header=False)
#                 buffer.seek(0)
#                 cur.copy("COPY {} FROM STDIN WITH CSV".format(table_name), buffer)
#                 conn.commit()
#     except Exception as e:
#         raise e
#     else:
#         print("Done.")